In [2]:
import numpy as np
import pandas as pd
train = pd.read_csv("train.csv")

test = pd.read_csv("test.csv")

In [3]:
Emap = {'S': 1, 'C': 2, 'Q': 3}
train.Embarked.loc[pd.isnull(train.Embarked)] = 'S'
datasets = [train, test]
for dataset in datasets:
    dataset["EmbarkedMap"] = dataset.Embarked.map(Emap)
CabinMap = {'NaN': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 8}
for dataset in datasets:
    dataset['HasCabin'] = 0
    dataset.HasCabin.loc[pd.notnull(dataset.Cabin)] = 1
    dataset["CMap"] = dataset.Cabin.str[0].map(CabinMap)
    dataset.CMap.fillna(0, inplace=True)
trainages = np.random.randint(train.Age.mean() - train.Age.std(), train.Age.mean() + train.Age.std(),
                              size=sum(pd.isnull(train.Age)))
testages = np.random.randint(test.Age.mean() - test.Age.std(), test.Age.mean() + test.Age.std(),
                             size=sum(pd.isnull(test.Age)))
train.Age.loc[pd.isnull(train.Age)] = trainages
test.Age.loc[pd.isnull(test.Age)] = testages
for dataset in datasets:
    dataset["Male"] = 0
    dataset.Male.loc[dataset.Sex == 'male'] = 1
    dataset["Title"] = dataset.Name.str.extract('( [A-Za-z]+)\.', expand=False)
d = dict(zip(np.unique(train.Title), np.arange(1, 18)))
for dataset in datasets:
    dataset['TMap'] = dataset.Title.map(d)
    dataset.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], 1, inplace=True)
    dataset.TMap.loc[dataset.TMap == 1] = 0
    dataset.TMap.loc[dataset.TMap == 2] = 0
    dataset.TMap.loc[dataset.TMap == 3] = 0
    dataset.TMap.loc[dataset.TMap == 4] = 0
    dataset.TMap.loc[dataset.TMap == 5] = 0
    dataset.TMap.loc[dataset.TMap == 6] = 0
    dataset.TMap.loc[dataset.TMap == 7] = 0
    dataset.TMap.loc[dataset.TMap == 8] = 0
    dataset.TMap.loc[dataset.TMap == 13] = 0
    dataset.TMap.loc[dataset.TMap == 14] = 0
    dataset.TMap.loc[dataset.TMap == 15] = 0
test.Fare.loc[pd.isnull(test.Fare)] = test.Fare.mean()
for dataset in datasets:
    dataset['Pclass1'] = 0
    dataset['Pclass2'] = 0
    dataset['Pclass3'] = 0
    dataset.Pclass1.loc[dataset.Pclass == 1] = 1
    dataset.Pclass2.loc[dataset.Pclass == 2] = 1
    dataset.Pclass3.loc[dataset.Pclass == 3] = 1
    dataset['EMap1'] = 0
    dataset['EMap2'] = 0
    dataset['EMap3'] = 0
    dataset.EMap1.loc[dataset.EmbarkedMap == 1] = 1
    dataset.EMap2.loc[dataset.EmbarkedMap == 2] = 1
    dataset.EMap3.loc[dataset.EmbarkedMap == 3] = 1
    dataset['TMap1'] = 0
    dataset['TMap2'] = 0
    dataset['TMap3'] = 0
    dataset['TMap4'] = 0
    dataset['TMap5'] = 0
    dataset.TMap1.loc[dataset.TMap == 0] = 1
    dataset.TMap2.loc[dataset.TMap == 9] = 1
    dataset.TMap3.loc[dataset.TMap == 10] = 1
    dataset.TMap4.loc[dataset.TMap == 11] = 1
    dataset.TMap5.loc[dataset.TMap == 12] = 1

/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
train.info()
from sklearn.model_selection import train_test_split
train1, train2 = train_test_split(train, test_size=0.20, random_state=33)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, criterion='entropy')
clf.fit(train.drop(['Survived', 'PassengerId', 'TMap', 'Title'], 1, inplace=False),train.Survived)
clf.score(train2.drop(['Survived', 'PassengerId', 'TMap', 'Title'], 1, inplace=False),train2.Survived)
train2_pred = clf.predict(
    train2.drop(['Survived', 'PassengerId', 'TMap', 'Title'], 1, inplace=False))
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(train2.Survived, train2_pred)
arr = pd.Series(clf.predict(test.drop(['PassengerId', 'TMap', 'Title'], 1, inplace=False)),name='Survived')
df = pd.concat([test.PassengerId, arr], axis=1)
df.to_csv('pred1.csv', sep=',', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 24 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
EmbarkedMap    891 non-null int64
HasCabin       891 non-null int64
CMap           891 non-null float64
Male           891 non-null int64
Title          891 non-null object
TMap           891 non-null int64
Pclass1        891 non-null int64
Pclass2        891 non-null int64
Pclass3        891 non-null int64
EMap1          891 non-null int64
EMap2          891 non-null int64
EMap3          891 non-null int64
TMap1          891 non-null int64
TMap2          891 non-null int64
TMap3          891 non-null int64
TMap4          891 non-null int64
TMap5          891 non-null int64
dtypes: float64(3), int64(20), object(1)
memory usage: 167.1+ KB
